# Convolutional Neural Network (CNN) - Image Classification

[![Open in Layer](https://development.layer.co/assets/badge.svg)](https://app.layer.ai/layer/image-classification) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/layerai/examples/blob/main/food-vision/food-vision.ipynb) [![Layer Examples Github](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/layerai/examples/tree/main/food-vision)


In [ ]:
!pip install layer -qqq

     |████████████████████████████████| 473 kB 4.4 MB/s 
     |████████████████████████████████| 40 kB 5.0 MB/s 
     |████████████████████████████████| 97 kB 4.4 MB/s 
     |████████████████████████████████| 256 kB 62.2 MB/s 
     |████████████████████████████████| 56 kB 3.3 MB/s 
     |████████████████████████████████| 1.3 MB 50.9 MB/s 
     |████████████████████████████████| 271 kB 43.2 MB/s 
     |████████████████████████████████| 2.4 MB 5.5 MB/s 
     |████████████████████████████████| 212 kB 41.3 MB/s 
     |████████████████████████████████| 132 kB 46.3 MB/s 
     |████████████████████████████████| 4.4 MB 33.5 MB/s 
     |████████████████████████████████| 381 kB 40.6 MB/s 
     |████████████████████████████████| 26.7 MB 1.1 MB/s 
     |████████████████████████████████| 159 kB 50.1 MB/s 
     |████████████████████████████████| 4.0 MB 9.6 MB/s 
     |████████████████████████████████| 16.8 MB 674 kB/s 
     |████████████████████████████████| 3.6 MB 29.4 MB/s 
     |█████████████████

In [ ]:
import layer
from layer.decorators import model, fabric,pip_requirements

In [ ]:
layer.login()

Please open the following link in your web browser. Once logged in, copy the code and paste it here.
https://auth.app.layer.ai/authorize?response_type=code&code_challenge=0giISng_VLMLjYh7dmbFihy5IQqyNrdo0Q7vL3QlQXk&code_challenge_method=S256&client_id=0STDdcnpK48P8A429EAAn93WNuLmViLR&redirect_uri=https://app.layer.ai/oauth/code&scope=offline_access&audience=https://app.layer.ai
Code: j51gD0vNLmhIBqDHRIkCMKOd3I7O3ad9PUHHMYy7WH_7H
Successfully logged into https://app.layer.ai


In [ ]:
layer.init("image-classification")

Project(name='image-classification', raw_datasets=[], derived_datasets=[], models=[], path=PosixPath('.'), project_files_hash='', readme='', account=Account(id=UUID('add1b570-c8e7-4187-b747-1d01104893a9'), name='layer'), _id=UUID('5505a21d-f3ef-4676-bf38-faf333790ee0'), functions=[])

In [ ]:
pip install wget

In [ ]:
@pip_requirements(packages=["wget","tensorflow","keras"])
@fabric("f-gpu-small")
@model(name="food-vision")
def train():
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.callbacks import EarlyStopping
    import os
    import matplotlib.pyplot as plt 
    from PIL import Image
    import numpy as np
    import pandas as pd
    import tarfile
    import wget
    wget.download("http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz")
    food_tar = tarfile.open('food-101.tar.gz')
    food_tar.extractall('.') 
    food_tar.close()
    plt.imshow(Image.open("food-101/images/beignets/2802124.jpg"))
    plt.axis('off')
    layer.log({"Sample image":plt.gcf()})
    base_dir = 'food-101/images'
    class_names = os.listdir(base_dir)
    train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   validation_split=0.2
                                   )
    validation_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
    image_size = (200, 200)
    training_set = train_datagen.flow_from_directory(base_dir,
                                                 seed=101,                                                 
                                                 target_size=image_size,
                                                 batch_size=32,
                                                 subset = "training",
                                                 class_mode='categorical')
    validation_set = validation_gen.flow_from_directory(base_dir, 
                                               target_size=image_size,
                                               batch_size=32, 
                                               subset = "validation",
                                               class_mode='categorical')
    model = Sequential([
            
    Conv2D(filters=32,kernel_size=(3,3),  input_shape = (200, 200, 3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    Conv2D(filters=32,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Conv2D(filters=64,kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.25),
    Dense(101, activation='softmax')])
    model.compile(optimizer='adam',
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.CategoricalAccuracy()])
    callback = EarlyStopping(monitor='loss', patience=3)
    epochs=10
    history = model.fit(training_set,validation_data=validation_set, epochs=epochs,callbacks=[callback])
    metrics_df = pd.DataFrame(history.history)
    layer.log({"Metrics":metrics_df})
    loss, accuracy = model.evaluate(validation_set)
    layer.log({"Accuracy on test dataset":accuracy})
    metrics_df[["loss","val_loss"]].plot()
    layer.log({"Loss plot":plt.gcf()})
    metrics_df[["categorical_accuracy","val_categorical_accuracy"]].plot()
    layer.log({"Accuracy plot":plt.gcf()})
    return model

In [ ]:
# Run Layer infra
layer.run([train],debug=True)

In [ ]:
# Run locally
train()

In [ ]:
from keras.preprocessing import image
import numpy as np
image_model = layer.get_model('layer/image-classification/models/food-vision').get_train()
!wget --no-check-certificate \
    https://upload.wikimedia.org/wikipedia/commons/b/b1/Buttermilk_Beignets_%284515741642%29.jpg \
    -O /tmp/Buttermilk_Beignets_.jpg

In [ ]:
test_image = image.load_img('/tmp/Buttermilk_Beignets_.jpg', target_size=(200, 200))

In [ ]:
test_image = image.img_to_array(test_image)


In [ ]:
test_image = test_image / 255.0

In [ ]:
test_image.shape

In [ ]:
test_image = np.expand_dims(test_image, axis=0)


In [ ]:
prediction = image_model.predict(test_image)


In [ ]:
prediction[0][0]

In [ ]:
import tensorflow as tf
scores = tf.nn.softmax(prediction[0])
scores = scores.numpy()

In [ ]:
import os
base_dir = 'food-101/images'
class_names = os.listdir(base_dir)
f"{class_names[np.argmax(scores)]} with a { (100 * np.max(scores)).round(2) } percent confidence." 